# READ DATA

In [ ]:
import pandas as pd
df = pd.read_csv(
    "https://www.dropbox.com/scl/fi/x61ewvoj9spslqblqweke/stocks_bonds_gold.csv?rlkey=ssmouraepasrj95b4rxj26bpk&dl=1"
)

# COMPUTE HISTORICAL MEANS, STD DEVS, AND COVS

In [ ]:
mn = df.mean()
sd = df.std()
cov = df.cov()

In [ ]:
import numpy as np
from cvxopt import matrix as Matrix
from cvxopt.solvers import qp as Solver
from cvxopt.solvers import options as SolverOptions
SolverOptions['show_progress'] = False

def optimal_s(rs, rb, means, cov, raver):
    n = len(means)
    Q = np.zeros((n + 2, n + 2))
    Q[2:, 2:] = raver * cov
    Q = Matrix(Q, tc="d")                               # quadratic form that penalizes variance
    p = np.array([-rs, rb] + list(-np.array(means)))
    p = Matrix(p, (len(p), 1), tc="d")                  # coefficients of objective function
    g = np.array([1, -1] + n * [1])
    a = np.zeros((2, n + 2))
    a[0, 0] = a[1, 1] = -1
    G = Matrix(np.vstack([a, g]))                       # coefficients of constraints
    h = np.array(2 * [0] + [1])
    h = Matrix(h, (len(h), 1), tc="d")                  # right-hand side of constraints
    sol = Solver(Q, p, G, h)
    return sol['x'] if sol['status']=='optimal' else None

def optimal_ns(rs, rb, means, cov, raver) :
    n = len(means)
    Q = np.zeros((n+2, n+2))
    Q[2:,2:] = raver*cov
    Q = Matrix(Q, tc='d')                                             # quadratic form that penalizes variance
    p = np.array([-rs, rb] + list(-np.array(means)))
    p = Matrix(p, (len(p), 1), tc='d')                                # coefficients of objective function
    g = np.array([1, -1] + n*[1])
    G = Matrix(np.vstack([-np.identity(n+2), g]), tc='d')             # coefficients of constraints
    h = np.array((n+2)*[0] + [1])
    h = Matrix(h, (len(h), 1), tc='d')                                # right-hand side of constraints
    sol = Solver(Q, p, G, h)
    return sol['x'] if sol['status']=='optimal' else None


In [1]:
import ssl
import pandas as pd

# Get returns data
ssl._create_default_https_context = ssl._create_unverified_context
file = "http://www.stern.nyu.edu/~adamodar/pc/datasets/histretSP.xls"
sheet = "Returns by year"
df = pd.read_excel(file, sheet_name=sheet, skiprows=16, header=[0, 1])
col1 = ("Unnamed: 0_level_0", "Year")
indx = df[df[col1].isna()].index[0]
df = df.iloc[:indx].set_index(col1)
df.index.name = "Year"
df = df["Annual Returns on Investments in"].iloc[:,:4]
df.columns = ["Stocks", "bills", "Treasuries", "Corporates"]
df = df.drop(columns=["bills"])


In [2]:
df.head()

Annual Returns on Investments in                            \
         S&P 500 (includes dividends) 3-month T.Bill US T. Bond   
Year                                                              
1928                         0.438112         0.0308   0.008355   
1929                        -0.082979         0.0316   0.042038   
1930                        -0.251236         0.0455   0.045409   
1931                        -0.438375         0.0231  -0.025589   
1932                        -0.086424         0.0107   0.087903   

                                                \
      Baa Corporate Bond Real Estate     Gold*   
Year                                             
1928            0.032196    0.014911  0.000969   
1929            0.030179   -0.020568 -0.001452   
1930            0.005398   -0.043000  0.000969   
1931           -0.156808   -0.081505 -0.173850   
1932            0.235896   -0.104664  0.212778   

     Value of $100 invested at start of 1928 in                              \
                  S&P 500 (includes dividends)3 3-month T.Bill4 US T. Bond5   
Year                                                                          
1928                                 143.811155      103.080000  100.835471   
1929                                 131.877782      106.337328  105.074397   
1930                                  98.745288      111.175676  109.845753   
1931                                  55.457774      113.743835  107.034958   
1932                                  50.664911      114.960894   116.44366   

                           ... Annual Risk Premium                         \
      Baa Corporate Bond2  ...      Stocks - Bonds Stocks - Baa Corp Bond   
Year                       ...                                              
1928           103.219551  ...            0.429757               0.405916   
1929           106.334569  ...           -0.125018              -0.113158   
1930           106.908543  ...           -0.296646              -0.256634   
1931            90.144455  ...           -0.412787              -0.281568   
1932           111.409173  ...           -0.174327              -0.322320   

                             Annual Real Returns  \
     Historical risk premium      Inflation Rate   
Year                                               
1928                     NaN           -0.011561   
1929                     NaN            0.005848   
1930                     NaN           -0.063953   
1931                     NaN           -0.093168   
1932                     NaN            -0.10274   

                                                                           \
     S&P 500 (includes dividends)2 3-month T. Bill (Real) !0-year T.Bonds   
Year                                                                        
1928                      0.454932               0.042856        0.020148   
1929                     -0.088311               0.025602         0.03598   
1930                     -0.200079               0.116932        0.116835   
1931                     -0.380674               0.128213        0.074522   
1932                      0.018184               0.126429        0.212472   

                                            
     Baa Corp Bonds Real Estate3      Gold  
Year                                        
1928       0.044268     0.026781  0.012676  
1929       0.024189    -0.026262 -0.007258  
1930        0.07409     0.022385  0.069359  
1931      -0.070178     0.012861 -0.088971  
1932       0.377411    -0.002145  0.351646  

[5 rows x 23 columns]

In [3]:
import quandl
quandl.ApiConfig.api_key = "f-5zoU2G4zzHaUtkJ7BY"


d = quandl.get("LBMA/GOLD")['USD (AM)']
gold = d.resample('Y').last().iloc[:-1]
gold.index = [x.year for x in gold.index]
gold.loc[1967] = d.iloc[0]
gold = gold.sort_index().pct_change().dropna()
gold.name = 'Gold'

In [4]:
df = df.join(gold, how="inner")

In [7]:
df.index.name="Year"
df.head()

,Stocks,Treasuries,Corporates,Gold
Year,,,,
1968,0.108149,0.032746,0.048451,0.192439
1969,-0.082414,-0.050140,-0.020252,-0.160667
1970,0.035611,0.167547,0.056496,0.061630
1971,0.142212,0.097869,0.140015,0.166934
1972,0.187554,0.028184,0.114091,0.483265


In [5]:
df.describe()

,Stocks,Treasuries,Corporates,Gold
count,55.000000,55.000000,55.000000,55.000000
mean,0.113763,0.064922,0.085898,0.100861
std,0.169278,0.098747,0.083688,0.267740
min,-0.365523,-0.178282,-0.144883,-0.321459
25%,0.017386,0.009874,0.031440,-0.039093
50%,0.142212,0.044907,0.097239,0.056843
75%,0.232560,0.139722,0.137667,0.206386
max,0.371952,0.328145,0.290525,1.334076


In [10]:
df.to_csv(
    "C:/users/kerry/Dropbox/BBCX Investments Textbook/site_files_other/stocks_bonds_gold.csv",
 )

In [ ]:
import ssl
import pandas as pd

# Get returns data
ssl._create_default_https_context = ssl._create_unverified_context
file = "http://www.stern.nyu.edu/~adamodar/pc/datasets/histretSP.xls"
sheet = "Returns by year"
df = pd.read_excel(file, sheet_name=sheet, skiprows=16, header=[0, 1])
col1 = ("Unnamed: 0_level_0", "Year")
indx = df[df[col1].isna()].index[0]
df = df.iloc[:indx].set_index(col1)
df.index.name = "Year"

df = df["Annual Real Returns on"].copy()
df = df[df.columns[1:5]]
df.columns = ["S&P 500", "TBills", "Treasuries", "Corporates"]
df.dtype = "float"
df.index = [int(x) for x in df.index]
df.index.name = "Year"
df = df.drop(columns=["TBills"])

# displaying the first 5 rows of df
df.head()

/Users/kailiao/PycharmProjects/appcode_draft/venv/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


,S&P 500,Treasuries,Corporates
Year,,,
1928,0.454932,0.020148,0.044268
1929,-0.088311,0.035980,0.024189
1930,-0.200079,0.116835,0.074090
1931,-0.380674,0.074522,-0.070178
1932,0.018184,0.212472,0.377411
1933,0.488460,0.010836,0.121111
1934,-0.026634,0.063520,0.170431
1935,0.424871,0.014439,0.100235
1936,0.300585,0.035176,0.097926


In [ ]:
# solutions
means = df.mean().to_numpy()
cov = df.cov().to_numpy()
rs = 0.02                             # example savings rate
rb = 0.06                             # example borrowing rate
raver = 2                             # example risk aversion
sol = optimal_ns(rs, rb, means, cov, raver)


if sol:
    sol = list(sol)
    print('Without Short Sales:')
    print(f'optimal amount to save is {sol[0]:.1%}')
    print(f'optimal amount to borrow is {sol[1]:.1%}')
    print(f'optimal investment in asset S&P 500 is {sol[2]:.1%}')
    print(f'optimal investment in asset Treasuries is {sol[3]:.1%}')
    print(f'optimal investment in asset Corporates is {sol[4]:.1%}')

Without Short Sales:
optimal amount to save is 0.0%
optimal amount to borrow is 0.0%
optimal investment in asset S&P 500 is 73.4%
optimal investment in asset Treasuries is 0.0%
optimal investment in asset Corporates is 26.6%


In [ ]:
# solutions
means = df.mean().to_numpy()
cov = df.cov().to_numpy()
rs = 0.02                             # example savings rate
rb = 0.06                             # example borrowing rate
raver = 2                             # example risk aversion
sol = optimal_s(rs, rb, means, cov, raver)


if sol:
    sol = list(sol)
    print('With Short Sales:')
    print(f'optimal amount to save is {sol[0]:.1%}')
    print(f'optimal amount to borrow is {sol[1]:.1%}')
    print(f'optimal investment in asset S&P 500 is {sol[2]:.1%}')
    print(f'optimal investment in asset Treasuries is {sol[3]:.1%}')
    print(f'optimal investment in asset Corporates is {sol[4]:.1%}')

With Short Sales:
optimal amount to save is 0.0%
optimal amount to borrow is -0.0%
optimal investment in asset S&P 500 is 58.4%
optimal investment in asset Treasuries is -122.7%
optimal investment in asset Corporates is 164.3%
